# Read an input JSON statement

> The methods in this notebook implement the functionalities for reading an xAPI statement stored in JSON format.
Statements are usually provided in the form of ```csv``` files, but in case you want to check all the metadata of a statement, you can directly work with the ```json``` version of a statement.
Here we show how to import the data and parse the json file to collect all the relevant information.

In [ ]:
#| default_exp input_json

In [ ]:
#| hide
from nbdev.showdoc import *

The libraries used to import the data:

In [ ]:
import json
from typing import Union, List
from datetime import datetime, timedelta
from pathlib import Path
from fastcore.test import *

As an example, in this package we provide a ```json``` file containing a statement and all its related metadata

In [ ]:
json_file = '../example_single_statement.json'

#### Load an xAPI statement 
Let's start parsing the json file

In [ ]:
#| export
def load_statement(json_file: str # Filename of the json containing the statement
                  ) -> dict: # A dictionary representing the statement structure
    """
    Load a json from file and store the information in a Python dictionary object.
    If the file does not exist, returns an empty dict and print an error message
    """
    if Path(json_file).exists():
        with open(json_file) as f:
            return json.load(f)
    else:
        print("ERROR: The specified file does not exist")
        return dict()

In [ ]:
my_statement = load_statement(json_file)

In [ ]:
#| export
def pretty_print_statement(statement: dict, # the statement dict imported from JSON
                          indent: int=4 # indentation used when printing
                          ) -> None:
    """
    Displays the content of the statement in a human readable format
    """
    print(json.dumps(statement, indent=indent))

In [ ]:
sample_json = json.loads('["foo", {"bar": ["baz", null, 1.0, 2]}]')
pretty_print_statement(sample_json, indent=2)

[
  "foo",
  {
    "bar": [
      "baz",
      null,
      1.0,
      2
    ]
  }
]


In [ ]:
#| export
def get_value(statement: dict, # Our xAPI statement imported from JSON
              key: str # The key we are interested in
             ) -> Union[str, dict, List, None]: # The value associated to the key in the statement
    """
    Return the value associated to the specified key in the statement dictionary.
    If the key does not exist, returns None
    """
    if key in statement:
        return statement[key]
    else:
        return None

In [ ]:
test_eq(get_value(my_statement, "not_a_key"), None)
test_eq(get_value(my_statement, "stored"), "2022-09-30T13:34:35.959Z")

#### Extract statement data
The following methids are used to extract the **actor**, **verb** and **object** information, which represents the core information provided in each statement, as well as the version statement

In [ ]:
#| export
def get_actor(statement: dict, # Our xAPI statement imported from JSON
             ) -> dict: # dictionary containing actor information
    """
    Extract the actor information from the statement
    """
    st = get_value(statement, "statement")
    #pretty_print_statement(st["actor"], indent = 2)
    return st["actor"]

In [ ]:
test_actor = {
      "objectType": "Agent",
      "name": "1s1116",
      "mbox": "mailto:student@app.com"
    }
actor = get_actor(my_statement)
test_eq(actor["objectType"], test_actor["objectType"])
test_eq(actor["name"], test_actor["name"])
test_eq(actor["mbox"], test_actor["mbox"])

In [ ]:
#| export
def get_actor_name(statement: dict, # Our xAPI statement imported from JSON
             ) -> str: # name of the actor
    """
    Quick access to the name field of the actor, as it is the most relevant information
    """
    st = get_value(statement, "statement")
    return st["actor"]["name"]

In [ ]:
test_eq(get_actor_name(my_statement), "1s1116")

In [ ]:
#| export
def get_verb(statement: dict, # Our xAPI statement imported from JSON
             ) -> dict: # dictionary containing verb information
    """
    Extract the verb information from the statement
    """
    st = get_value(statement, "statement")
    return st["verb"]

In [ ]:
test_verb = {
      "id": "http://id.tincanapi.com/verb/selected/",
      "display": {
        "en-US": "Selected"
      }
}
verb = get_verb(my_statement)
test_eq(verb["id"], test_verb["id"])
test_eq(verb["display"]["en-US"], test_verb["display"]["en-US"])

In [ ]:
#| export
def get_verb_str(statement: dict, # Our xAPI statement imported from JSON
             ) -> str: # the displayed verb
    """
    Quick access to the display field of the verb, as it is the most relevant information
    """
    st = get_value(statement, "statement")
    return st["verb"]["display"]["en-US"]

In [ ]:
test_eq(get_verb_str(my_statement), "Selected")

In [ ]:
#| export
def get_object(statement: dict, # Our xAPI statement imported from JSON
             ) -> dict: # dictionary containing object information
    """
    Extract the object information from the statement
    """
    st = get_value(statement, "statement")
    return st["object"]

In [ ]:
test_obj = {
      "objectType": "Activity",
      "id": "http://example.com/activities/student-lesson",
      "definition": {
        "name": {
          "en-US": "Lesson"
        },
        "description": {
          "en-US": "Level 1 Module8 started"
        }
      }
}
obj = get_object(my_statement)
test_eq(obj["id"], test_obj["id"])
test_eq(obj["objectType"], test_obj["objectType"])
test_eq(obj["definition"]["name"]["en-US"], test_obj["definition"]["name"]["en-US"])
test_eq(obj["definition"]["description"]["en-US"], test_obj["definition"]["description"]["en-US"])    

In [ ]:
#| export
def get_object_definition(statement: dict, # Our xAPI statement imported from JSON
             ) -> str: # the object definition
    """
    Quick access to the object definition
    """
    st = get_value(statement, "statement")
    return st["object"]["definition"]["name"]["en-US"]

In [ ]:
test_eq(get_object_definition(my_statement), "Lesson")

In [ ]:
#| export
def get_object_description(statement: dict, # Our xAPI statement imported from JSON
             ) -> str: # the object description
    """
    Quick access to the object description
    """
    st = get_value(statement, "statement")
    return st["object"]["definition"]["description"]["en-US"]

In [ ]:
test_eq(get_object_description(my_statement), "Level 1 Module8 started")

#### Extract the metadata information
The following methods are used to extract the metadata fields we may be interested in

##### Get general metadata information

In [ ]:
#| export
def get_stored(statement: dict, # Our xAPI statement imported from JSON
              ) -> datetime: # datetime object representing the time the statement was stored in the database
    """
    Extract the date and time information of when the statement was stored in the database
    """
    stored_str = get_value(statement, "stored")
    return datetime.strptime(stored_str, "%Y-%m-%dT%H:%M:%S.%f%z")

In [ ]:
my_date = datetime.strptime("2022-09-30T13:34:35.959Z", "%Y-%m-%dT%H:%M:%S.%f%z")
test_eq(get_stored(my_statement), my_date)

In [ ]:
#| export
def get_timestamp(statement: dict, # Our xAPI statement imported from JSON
              ) -> datetime: # datetime object representing the time the statement was generated
    """
    Extract the date and time information of when the statement was created
    """
    timestamp_str = get_value(statement, "timestamp")
    return datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S.%f%z")

In [ ]:
my_ts = datetime.strptime("2022-09-30T13:34:35.959Z", "%Y-%m-%dT%H:%M:%S.%f%z")
test_eq(get_timestamp(my_statement), my_date)

In [ ]:
#| export
def get_time_diff(statement: dict, # Our xAPI statement imported from JSON
                 ) -> timedelta:   # Time difference between when the statement was sent and when it was stored
    """
    Compute the time difference between when a statement was sent and when it was stored in the database
    """
    ts_sent = get_timestamp(statement)
    ts_stored = get_stored(statement)
    return ts_stored - ts_sent

In [ ]:
test_eq(get_time_diff(my_statement), timedelta()) # In our example statement the timestamps are the same

##### Get Boolean metadata information
These methods return the metadata providing boolean information related to the statement

In [ ]:
#| export
def is_active(statement: dict, # Our xAPI statement imported from JSON
              ) -> bool: # Boolean representive whether active or not
    """
    Extract the Active field from the statement
    """
    return get_value(statement, "active")

In [ ]:
test_eq(is_active(my_statement), True)

In [ ]:
#| export
def is_voided(statement: dict, # Our xAPI statement imported from JSON
              ) -> bool: # Boolean representive whether statement is voided or not
    """
    Extract the Active field from the statement
    """
    return get_value(statement, "voided")

In [ ]:
test_eq(is_voided(my_statement), False)

In [ ]:
#| export
def has_generated_id(statement: dict, # Our xAPI statement imported from JSON
              ) -> bool: # Boolean representive whether statement has generated id
    """
    Extract the Active field from the statement
    """
    return get_value(statement, "hasGeneratedId")

In [ ]:
test_eq(has_generated_id(my_statement), False)

##### Get ID metadata information
These methods return the metadata providing ID information

In [ ]:
#| export
def get_client(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # ID of the client
    """
    Extract the client field from the statement
    """
    return get_value(statement, "client")

In [ ]:
test_eq(get_client(my_statement), "60ffcf8d448b2d059a63e3c4")

In [ ]:
#| export
def get_LRS(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # ID of the Learning Record Store
    """
    Extract the Learning Record Store ID field from the statement
    """
    return get_value(statement, "lrs_id")

In [ ]:
test_eq(get_LRS(my_statement), "60ffcf8d448b2d059a63e3c3")

In [ ]:
#| export
def get_id(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # ID of the statement
    """
    Extract the ID field from the statement
    """
    return get_value(statement, "_id")

In [ ]:
test_eq(get_id(my_statement), "6336f06c6ce79d05ebef40a7")

In [ ]:
#| export
def get_persona_id(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # id of the persona associated to the statement
    """
    Extract the persona identifier
    """
    return get_value(statement, "personaIdentifier")

In [ ]:
test_eq(get_persona_id(my_statement), "6103e17eaed02c30c695bffb")

In [ ]:
#| export
def get_organisation(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # id of the organization to the statement
    """
    Extract the persona identifier
    """
    return get_value(statement, "organisation")

In [ ]:
test_eq(get_organisation(my_statement), "60faab70448b2d059a63e375")

In [ ]:
#| export
def get_hash(statement: dict, # Our xAPI statement imported from JSON
              ) -> str: # hash of the statement
    """
    Extract the hash
    """
    return get_value(statement, "hash")

In [ ]:
test_eq(get_hash(my_statement), "3268dd76c35a6077796979e0613654ecf449c46e")

##### Get queues metadata information
These methods return the metadata related to the queues information in the statement

In [ ]:
#| export
def get_completed_fw_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of completed forwarding queues in the statement
    """
    Extract the List of completed forwarding queues in the statement
    """
    return get_value(statement, "completedForwardingQueue")

In [ ]:
test_eq(get_completed_fw_queues(my_statement), list())

In [ ]:
#| export
def get_failed_fw_log(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of failed forwarding log messages in the statement
    """
    Extract the List of failed forwarding log messages in the statement
    """
    return get_value(statement, "failedForwardingLog")

In [ ]:
test_eq(get_failed_fw_log(my_statement), list())

In [ ]:
#| export
def get_completed_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of completed queues in the statement
    """
    Extract the List of completed queues in the statement
    """
    return get_value(statement, "completedQueues")

In [ ]:
COMPL_QS = ["STATEMENT_FORWARDING_QUEUE", "STATEMENT_PERSON_QUEUE", "STATEMENT_QUERYBUILDERCACHE_QUEUE"]
test_eq(get_completed_queues(my_statement), COMPL_QS)

In [ ]:
#| export
def get_completed_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of completed queues in the statement
    """
    Extract the List of completed queues in the statement
    """
    return get_value(statement, "completedQueues")

In [ ]:
test_eq(get_completed_fw_queues(my_statement), list())

In [ ]:
#| export
def get_dead_forwarding_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of dead forwarding queues in the statement
    """
    Extract the List of dead forwarding queues in the statement
    """
    return get_value(statement, "deadForwardingQueue")

In [ ]:
test_eq(get_dead_forwarding_queues(my_statement), list())

In [ ]:
#| export
def get_pending_forwarding_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of completed queues in the statement
    """
    Extract the List of pending forwarding queues in the statement
    """
    return get_value(statement, "pendingForwardingQueue")

In [ ]:
test_eq(get_pending_forwarding_queues(my_statement), list())

In [ ]:
#| export
def get_processing_queues(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of processing queues in the statement
    """
    Extract the List of processing queues in the statement
    """
    return get_value(statement, "processingQueues")

In [ ]:
test_eq(get_processing_queues(my_statement), list())

In [ ]:
#| export
def get_registrations(statement: dict, # Our xAPI statement imported from JSON
                ) -> List: # List of registrations in the statement
    """
    Extract the List of registrations in the statement
    """
    return get_value(statement, "registrations")

In [ ]:
test_eq(get_registrations(my_statement), list())

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()